In [1]:
import os

os.chdir('../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

ICM_all_dataframe = pd.read_csv("Dataset/data_ICM_metadata.csv")
URM_all_dataframe = pd.read_csv("Dataset/data_train.csv")

In [4]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout

ICM_all = sps.coo_matrix((ICM_all_dataframe['data'], (ICM_all_dataframe['item_id'], ICM_all_dataframe['feature_id'])))
ICM_all = ICM_all.tocsr()

URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

#ICM_train_validation, ICM_test = split_train_in_two_percentage_global_sample(ICM_all, train_percentage=0.8)
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [7]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

stacked_URM = sps.vstack([URM_all, ICM_all.T])
# RP3
rp3_recommender = RP3betaRecommender(stacked_URM)
rp3_recommender.fit(topK=14, alpha=0.3683550822991944, beta=0.19877125816137325)

RP3betaRecommender: Similarity column 38121 (100.0%), 914.18 column/sec. Elapsed time 41.70 sec


In [8]:
target_user = pd.read_csv("Dataset/data_target_users_test.csv")
print(target_user.head(10))

user_id = target_user['user_id']
recommendations = []
for user in user_id:
    recommendations.append(rp3_recommender.recommend(user, cutoff=10))

# Saving in a submission file for the competition
submission = pd.DataFrame(columns=['user_id', 'item_list'])
submission['user_id'] = user_id
# Convert list of recommendations into a space-separated string
submission['item_list'] = [' '.join(map(str, recs)) for recs in recommendations]

# Save the submission file
submission.to_csv(f"submissions/rp3_stacked_samebestparam.csv", index=False)

   user_id
0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9
